In [1]:
import sys
!{sys.executable} -m pip install pyspark python-dotenv minio

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env') # đi ra ngoài 1 cấp để tìm file .env

# Cấu hình Spark Session
from pyspark.sql import SparkSession

access_key = os.getenv("MINIO_ACCESS_key")
secret_key = os.getenv("MINIO_SECRET_key")

spark = (
    SparkSession.builder.appName("SpotifyEDA")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", access_key)
    .config("spark.hadoop.fs.s3a.secret.key", secret_key)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4")
    .getOrCreate()
)

print("Spark Session is ready!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.8 MB/s eta 0:00:00ta 0:00:01
Spark Session is ready!


In [ ]:
raw_data_path = "s3a://spotify-raw-data/mpd.slice.0-999.json"

df = spark.read.json(raw_data_path)

df.printSchema()

df.show(5)

In [ ]:
from pyspark.sql.functions import explode, col

df_playlists = df.select(explode("playlists").alias("playlist_details"))

df_playlists.printSchema()

df_playlists.select("playlist_details.*").show(5)